# Global Daily SST Analysis: Identifying Marine Extremes with `MarEx-Detect`

**MarEx-Detect** transforms raw oceanographic time series into standardised extreme event boolean datasets through a four-stage processing pipeline optimised for **exascale climate data**.

---

## Processing Stages

### 1. Anomaly Computation
Choose between 4 methodological approaches for calculating **anomalies**:

**Shifting Baseline** *(scientifically rigorous)*
- Computes anomalies relative to backward-looking rolling climatology using previous N years of data
- More accurate representation of baseline conditions but computationally expensive
- Shortens usable time series by window duration

**Harmonic Detrend** *(computationally efficient)*
- Removes trends and seasonal cycles using harmonic decomposition (mean + annual/semi-annual harmonics + polynomial trends)
- Efficient processing but may introduce statistical biases in certain metrics

**Fixed Baseline** *(simple interpretation, limited usefulness)*
— Removes daily climatology using the full time series without detrending
- Preserves long-term trends and maintains a simple interpretation

**Detrend Fixed Baseline** *(efficient, accurate, and data-preserving)*
- Polynomial detrending followed by removal of the fixed daily climatology
- Removes long-term trends and maintains the full time series of data

### 2. Threshold Calculation
Define percentile thresholds for extreme event detection:

**Hobday Extreme** *(methodologically accurate)*  
- Calculates day-of-year specific thresholds using local temporal windows
- In line with Hobday et al. (2016) methodology for marine heatwave detection of a single time-series
- Memory-intensive but captures seasonal variability evolution

**Global Extreme** *(computationally optimised)*  
- Applies single*percentile threshold computed globally across all time points
- Fast computation but may not capture seasonality evolution

### 3. Extreme Value Identification
Deploy percentile-based detection algorithms:
- Exact percentiles for precision or PDF-based histogram approximation for efficiency
- Optimised for massive datasets using vectorised operations, `dask` distributed computation, and `flox` for efficient grouping

### 4. Output Assembly & Optimisation
Prepare data for downstream tracking algorithms:
- Intelligent chunking optimises Dask array patterns for parallel processing
- Zarr serialisation enables efficient distributed I/O operations

---

## Performance Benchmark

### Shifting-Baseline & Hobday Extreme Definition
70-year global daily analysis at 5km resolution on the _unstructured grid_  (~15M grid points):
- **Completion time**: ~30 minutes using 512 cores
- **Scalability**: Linear scaling to 2048+ cores demonstrated

In [1]:
from getpass import getuser
from pathlib import Path

import dask
import intake
import numpy as np
import xarray as xr

import marEx
import marEx.helper as hpc

In [ ]:
# Lustre Scratch Directory
scratch_dir = Path("/scratch") / getuser()[0] / getuser()

In [3]:
# Start Distributed Dask Cluster
client = hpc.start_distributed_cluster(
    n_workers=512,
    workers_per_node=64,
    runtime=29,
    node_memory=512,
    scratch_dir=scratch_dir / "clients",
)  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmpsvja5opp'
Memory per Worker: 8.00 GB
Hostname: l40204
Forward Port: l40204:8889
Dashboard Link: localhost:8889/status


In [4]:
# Import 40 years of Daily Native-Grid ICON data (ref. EERIE project)

cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
expid = "eerie-control-1950"
version = "v20240618"
model = "icon-esm-er"
gridspec = "native"

dat = cat["dkrz.disk.model-output"][model][expid][version]["ocean"][gridspec]

In [ ]:
# Load Data
sst = dat["2d_daily_mean"](chunks={}).to_dask().to.isel(depth=0).drop_vars({"depth", "cell_sea_land_mask"})

In [ ]:
# Load the grid & neighbours

grid2d = dat["2d_grid"](chunks={}).to_dask().rename({"cell": "ncells"})
neighbours = grid2d.neighbor_cell_index.rename({"clat": "lat", "clon": "lon"})
areas = grid2d.cell_area.rename({"clat": "lat", "clon": "lon"})

In [ ]:
# Process Data using `MarEx Detect` helper functions:

extremes_ds = marEx.preprocess_data(
    sst,
    method_anomaly="shifting_baseline",  # Anomalies from a rolling climatology using previous window_year years -- more "correct", but shortens time series by window_year years
    method_extreme="hobday_extreme",  # Local day-of-year specific thresholds with windowing
    threshold_percentile=95,  # Use the 95th percentile as the extremes threshold
    window_year_baseline=15,
    smooth_days_baseline=21,  # Defines the rolling climatology window (15 years) and smoothing window (21 days) for determining the anomalies
    window_days_hobday=11,  # Defines the window (11 days) of compiled samples collected for the extremes detection
    dask_chunks={
        "time": 2
    },  # Dask chunks for *output* data (this is much smaller than the input chunks because the Tracking/ID is more memory-intensive)
    neighbours=neighbours,  # Pass information about neighbours to be used in subsequent processing
    cell_areas=areas,  # Pass information about each Unstructured Grid's cell area (in metres) to be used in subsequent processing
    dimensions={"time": "time", "x": "ncells"},
    coordinates={
        "time": "time",
        "x": "lon",
        "y": "lat",
    },  # Specify the names of the coordinates associated with the single unstructured spatial dimension
)  # Not specifying 'ydim' tells MarEx-Detect that it is an Unstructured Grid
extremes_ds

<xarray.Dataset> Size: 1TB
Dimensions:         (ncells: 14886338, time: 18262, nv: 3)
Coordinates:
    lat             (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    lon             (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
  * nv              (nv) int64 24B 0 1 2
  * time            (time) datetime64[ns] 146kB 1991-01-01T23:59:59 ... 2040-...
Dimensions without coordinates: ncells
Data variables:
    cell_areas      (ncells) float32 60MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    dat_anomaly     (time, ncells) float32 1TB dask.array<chunksize=(4, 14886338), meta=np.ndarray>
    extreme_events  (time, ncells) bool 272GB dask.array<chunksize=(4, 14886338), meta=np.ndarray>
    thresholds      (ncells, dayofyear) float32 12GB 0.0 0.0 ... 0.00255 0.0023
    mask            (ncells) bool 15MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    neighbours      (nv, ncells) int32 179MB dask.array<chunksize=(3, 14886338), meta=np.ndarray>
Attributes:
    method_anomaly:        shifting_baseline
    method_extreme:        hobday_extreme
    threshold_percentile:  95
    preprocessing_steps:   ['Rolling climatology using 15 years', 'Smoothed w...
    window_year_baseline:  15
    smooth_days_baseline:  21
    window_days_hobday:    11

In [ ]:
# Save data to `zarr` for more efficient parallel I/O

file_name = scratch_dir / "mhws" / "extremes_binary_unstruct_shifting_hobday.zarr"
extremes_ds.to_zarr(file_name, mode="w")